In [30]:
import pandas as pd
import numpy as np
import logging
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, recall_score, precision_score, f1_score,
    roc_auc_score, average_precision_score
)
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
import joblib

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (accuracy_score, 
                             recall_score, 
                             precision_score, 
                             f1_score, 
                             roc_auc_score, 
                             average_precision_score,
                             get_scorer)
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split


from xgboost import XGBClassifier

import mlflow
from mlflow.models import infer_signature
import mlflow.sklearn
from sklearn.metrics import get_scorer
from typing import Optional

from lazypredict.Supervised import LazyClassifier

### Setup logging

In [31]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s"
)
logger = logging.getLogger(__name__)
logger.info("Starting the data cleaning process...")

2025-07-29 00:30:25,074 [INFO] Starting the data cleaning process...


In [32]:
# Folder to store the cleaned data
data_folder = '../data/'

# Load positive samples -- lncRNAs
df_pos = pd.read_parquet(f'{data_folder}ncr_training_redux.parquet')


# Load negative samples -- protein coding genes
df_neg = pd.read_parquet(f'{data_folder}pcg_training_redux.parquet')

# Target column (Binary classification)
label = 'y'

In [33]:
# Drop columns that are not features
cols_not_features = ['chromosome', 'start', 'end', 'gene_name', 'sequence', 'ss_structure', 'cpat_orf_len', 'cpat_fickett_score', 'cpat_hexamer_score', 'cpat_cod_prob']

logging.info(f"Columns not used as features: {cols_not_features}")
df_pos = df_pos.drop(columns=cols_not_features)
df_neg = df_neg.drop(columns=cols_not_features)
init_pos_records = df_pos.shape[0]
init_neg_records = df_neg.shape[0]

2025-07-29 00:30:25,390 [INFO] Columns not used as features: ['chromosome', 'start', 'end', 'gene_name', 'sequence', 'ss_structure', 'cpat_orf_len', 'cpat_fickett_score', 'cpat_hexamer_score', 'cpat_cod_prob']


In [34]:
# Check if dfs can be merged
if df_pos.shape[1] != df_neg.shape[1]:
    logger.error("DataFrames have different number of columns after dropping non-feature columns.")
    raise ValueError("DataFrames have different number of columns after dropping non-feature columns.")
# Check if the columns are the same
if not all(df_pos.columns == df_neg.columns):
    logger.error("DataFrames have different columns after dropping non-feature columns.")
    raise ValueError("DataFrames have different columns after dropping non-feature columns.")

df = pd.concat([df_pos, df_neg])
del df_pos, df_neg
logger.info(f"DataFrames merged. Shape: {df.shape}")
logger.info(f"DataFrame columns: {df.columns.tolist()}")
logger.info(f"DataFrame dtypes: {df.dtypes}")

2025-07-29 00:30:25,411 [INFO] DataFrames merged. Shape: (33643, 37)
2025-07-29 00:30:25,412 [INFO] DataFrame columns: ['length', 'cov_epdnew', 'cov_h3k4me3', 'cov_remap', 'cov_s2_pol2', 'cov_tfbs', 'cov_tss_minus', 'cov_tss_plus', 'max_remap', 'max_s2_pol2', 'max_tfbs', 'max_tss_plus', 'mean_gc', 'mean_h3k4me3', 'mean_pcons27', 'mean_phylocons124', 'min_tss_minus', 'std__phylocons124', 'std_pcons27', 'sum__phylocons124', 'sum_h3k4me3', 'sum_pcons27', 'sum_s2_pol2', 'sum_tss_minus', 'sum_tss_plus', 'ss_mfe', '3mer_SVD1', '4mer_SVD1', '5mer_SVD1', '6mer_SVD1', '7mer_SVD1', '8mer_SVD1', '9mer_SVD1', '10mer_SVD1', '11mer_SVD1', '12mer_SVD1', 'y']
2025-07-29 00:30:25,413 [INFO] DataFrame dtypes: length                 int64
cov_epdnew           float64
cov_h3k4me3          float64
cov_remap            float64
cov_s2_pol2          float64
cov_tfbs             float64
cov_tss_minus        float64
cov_tss_plus         float64
max_remap            float64
max_s2_pol2          float64
max_tfbs 

In [35]:
# Remove duplicates
df = df.drop_duplicates(ignore_index=False)
logger.info(f"Duplicates removed. Shape: {df.shape}")

# Percentage of records dropped due to duplicates
duplicates_percentage = (1 - df.shape[0] / (init_pos_records + init_neg_records)) * 100
logger.info(f"Percentage of records dropped due to duplicates: {duplicates_percentage:.2f}%")

# Count NAs per each column
na_counts = df.isna().sum()
na_counts = na_counts[na_counts > 0]
if not na_counts.empty:
    logger.info(f"Columns with NAs: \n{na_counts}")
else:
    logger.info("No NAs found in the DataFrame.")

2025-07-29 00:30:25,449 [INFO] Duplicates removed. Shape: (32906, 37)
2025-07-29 00:30:25,450 [INFO] Percentage of records dropped due to duplicates: 2.19%
2025-07-29 00:30:25,452 [INFO] Columns with NAs: 
cov_epdnew             53
cov_h3k4me3          1075
cov_s2_pol2           661
cov_tss_minus         194
cov_tss_plus          215
max_s2_pol2           661
max_tss_plus          215
mean_h3k4me3         1075
mean_pcons27           36
mean_phylocons124       1
min_tss_minus         194
std__phylocons124       1
std_pcons27            36
sum__phylocons124      12
sum_h3k4me3          2222
sum_pcons27            63
sum_s2_pol2           740
sum_tss_minus         295
sum_tss_plus          293
dtype: int64
2025-07-29 00:30:25,450 [INFO] Percentage of records dropped due to duplicates: 2.19%
2025-07-29 00:30:25,452 [INFO] Columns with NAs: 
cov_epdnew             53
cov_h3k4me3          1075
cov_s2_pol2           661
cov_tss_minus         194
cov_tss_plus          215
max_s2_pol2          

In [36]:
# Logic for data cleaning:
# 1. If columns are UCSC (BigWig or BigBed) statistics features or CPAT scores (no ORFs found), fill NAs with 0
logic_op1 = (df.columns.str.startswith(tuple(['min_', 'max_', 'mean_', 'std_', 'sum_', 'cov_'])) | df.columns.str.startswith('cpat_'))
for col in df.columns:
    if col in df.columns[logic_op1]:
        if df[col].isna().sum() > 0:
            logger.info(f"Filling NAs in column {col} with 0")
            df[col] = df[col].fillna(0)

# 2. If columns are ss_* features, drop rows with NAs. This seems to be the best approach as no structure was calculated for the sequence
logic_op2 = df.columns.str.startswith('ss_')
for col in df.columns:
    if col in df.columns[logic_op2]:
        if df[col].isna().sum() > 0:
            logger.info(f"Dropping rows with NAs in column {col}")
            df = df.dropna(subset=[col])
            logger.info(f"Rows with NAs in column {col} dropped. Shape: {df.shape}")
        # Drop values where `ss_mfe` is not < 0
        if col == 'ss_mfe':
            df = df[df[col] < 0]
            logger.info(f"Rows with ss_mfe >= 0 dropped. Shape: {df.shape}")

# 3. If columns start with '0' or '1', drop rows with NAs. This seems to be the best approach as we have no counts for all required k-mers
logic_op3 = df.columns.str.startswith(('0', '1')) | df.columns.str.contains('mer_SVD')
for col in df.columns:
    if col in df.columns[logic_op3]:
        if df[col].isna().sum() > 0:
            logger.info(f"Dropping rows with NAs in column {col}")
            df = df.dropna(subset=[col])
            logger.info(f"Rows with NAs in column {col} dropped. Shape: {df.shape}")

# 4. If columns are categorical, fill NAs with ''
logic_op4 = df.dtypes == 'object'
for col in df.columns:
    if col in df.columns[logic_op4]:
        if df[col].isna().sum() > 0:
            logger.info(f"Filling NAs in column {col} with empty string")
            df[col] = df[col].fillna('')

2025-07-29 00:30:25,460 [INFO] Filling NAs in column cov_epdnew with 0
2025-07-29 00:30:25,462 [INFO] Filling NAs in column cov_h3k4me3 with 0
2025-07-29 00:30:25,463 [INFO] Filling NAs in column cov_s2_pol2 with 0
2025-07-29 00:30:25,464 [INFO] Filling NAs in column cov_tss_minus with 0
2025-07-29 00:30:25,465 [INFO] Filling NAs in column cov_tss_plus with 0
2025-07-29 00:30:25,467 [INFO] Filling NAs in column max_s2_pol2 with 0
2025-07-29 00:30:25,467 [INFO] Filling NAs in column max_tss_plus with 0
2025-07-29 00:30:25,469 [INFO] Filling NAs in column mean_h3k4me3 with 0
2025-07-29 00:30:25,469 [INFO] Filling NAs in column mean_pcons27 with 0
2025-07-29 00:30:25,462 [INFO] Filling NAs in column cov_h3k4me3 with 0
2025-07-29 00:30:25,463 [INFO] Filling NAs in column cov_s2_pol2 with 0
2025-07-29 00:30:25,464 [INFO] Filling NAs in column cov_tss_minus with 0
2025-07-29 00:30:25,465 [INFO] Filling NAs in column cov_tss_plus with 0
2025-07-29 00:30:25,467 [INFO] Filling NAs in column max

In [37]:
# Compare the current postive and negative instances with the ones before data cleaning
final_pos_records = df[df['y'] == 1].shape[0]
final_neg_records = df[df['y'] == 0].shape[0]
logger.info(f"Positive records before data cleaning: {init_pos_records}, after: {final_pos_records}")
logger.info(f"Negative records before data cleaning: {init_neg_records}, after: {final_neg_records}")
# Log percentage of positive and negative records lost
pos_records_lost = init_pos_records - final_pos_records
neg_records_lost = init_neg_records - final_neg_records
if init_pos_records > 0:
    logger.info(f"Percentage of positive records lost: {pos_records_lost / init_pos_records * 100:.2f}%")
if init_neg_records > 0:
    logger.info(f"Percentage of negative records lost: {neg_records_lost / init_neg_records * 100:.2f}%")

2025-07-29 00:30:25,498 [INFO] Positive records before data cleaning: 2981, after: 2981
2025-07-29 00:30:25,499 [INFO] Negative records before data cleaning: 30662, after: 29925
2025-07-29 00:30:25,500 [INFO] Percentage of positive records lost: 0.00%
2025-07-29 00:30:25,500 [INFO] Percentage of negative records lost: 2.40%
2025-07-29 00:30:25,499 [INFO] Negative records before data cleaning: 30662, after: 29925
2025-07-29 00:30:25,500 [INFO] Percentage of positive records lost: 0.00%
2025-07-29 00:30:25,500 [INFO] Percentage of negative records lost: 2.40%


## Multi-hot encoding
Perform multi-hot encoding on categorical features. The dataset has previously been filled with emtpy string `''` on NA values for the categorical features

In [38]:
# Multi-hot encoding
# Get a df with only the categorical features. These start with 'entries_'
df_categorical = df.loc[:, df.columns.str.startswith('entries_')]

vectorizer = CountVectorizer(tokenizer=lambda x: x.split(','), binary=True, token_pattern=None)

mhe_df = pd.DataFrame(index=df.index)

# For each categorical feature, perform multi-hot encoding
for col in df_categorical.columns:
    # Only fit on non-empty values to avoid creating a column for ''
    non_empty = df_categorical[col][df_categorical[col] != '']
    if not non_empty.empty:
        vectorizer.fit(non_empty)
        # Transform all values (including empty) to ensure shape matches
        encoded = vectorizer.transform(df_categorical[col])
        encoded_df = pd.DataFrame(encoded.toarray(), columns=vectorizer.get_feature_names_out(), index=df.index)
        # Convert to bool dtype
        encoded_df = encoded_df.astype(bool)
        mhe_df = pd.concat([mhe_df, encoded_df], axis=1)
    else:
        # If all values are empty, skip this column
        continue

In [39]:
mhe_df.shape

(32906, 0)

In [40]:
# Check if duplicated columns exist in the multi-hot encoded DataFrame
if mhe_df.columns.duplicated().any():
    logger.warning("Duplicated columns found in the multi-hot encoded DataFrame. Removing duplicates.")
    # Merge duplicated columns by taking the logical OR (future-proof way)
    mhe_df = mhe_df.T.groupby(level=0).any().T.astype(bool)

In [41]:
mhe_df.shape

(32906, 0)

In [42]:
# Concatenate the multi-hot encoded features with the original DataFrame. Making sure the indexes match
df = pd.concat([df.drop(columns=df_categorical.columns), mhe_df], axis=1)

In [43]:
# Move the 'y' column to the beginning of the DataFrame
y_col = df.pop(label)
df.insert(0, label, y_col)

In [44]:
df

,y,length,cov_epdnew,cov_h3k4me3,cov_remap,cov_s2_pol2,cov_tfbs,cov_tss_minus,cov_tss_plus,max_remap,...,3mer_SVD1,4mer_SVD1,5mer_SVD1,6mer_SVD1,7mer_SVD1,8mer_SVD1,9mer_SVD1,10mer_SVD1,11mer_SVD1,12mer_SVD1
FBtr0344106,True,1602,0.07,1.00,1.00,1.01,0.93,0.71,0.42,721.00,...,0.72,0.50,0.35,0.24,0.17,0.12,0.08,0.06,0.04,0.03
FBtr0336985,True,1190,0.00,1.00,1.00,1.00,0.95,0.46,0.28,542.00,...,0.71,0.50,0.35,0.25,0.18,0.12,0.09,0.06,0.05,0.03
FBtr0345955,True,654,0.00,1.00,1.00,0.85,0.95,0.03,0.06,299.00,...,0.71,0.50,0.35,0.25,0.17,0.12,0.08,0.06,0.04,0.02
FBtr0345962,True,259,0.00,1.00,1.00,1.00,0.90,0.05,0.13,322.00,...,0.70,0.50,0.36,0.26,0.19,0.13,0.10,0.07,0.05,0.04
FBtr0347040,True,2215,0.00,1.00,0.94,0.55,0.96,0.09,0.07,447.00,...,0.70,0.50,0.36,0.25,0.18,0.13,0.09,0.06,0.04,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FBtr0346773,False,45929,0.00,0.00,0.32,0.00,0.95,0.00,0.00,155.00,...,0.70,0.50,0.35,0.25,0.18,0.13,0.09,0.07,0.05,0.04
FBtr0346673,False,1286,0.00,0.00,1.00,0.00,0.97,0.00,0.00,96.00,...,0.70,0.50,0.36,0.25,0.18,0.13,0.10,0.07,0.05,0.04
FBtr0347450,False,309,0.00,0.00,0.00,0.00,0.96,0.00,0.00,0.00,...,0.71,0.50,0.35,0.25,0.17,0.12,0.09,0.06,0.04,0.03
FBtr0347429,False,402,0.00,0.00,0.00,0.00,0.96,0.00,0.00,0.00,...,0.71,0.50,0.35,0.25,0.18,0.13,0.09,0.06,0.05,0.03


In [45]:
# Substitute forbidden characters in column names
df.columns = df.columns.str.replace(r'[\[\]<>,]', '_', regex=True)

In [46]:
df_pos = df[df['y'] == 1]
df_neg = df[df['y'] == 0]
# Save the cleaned DataFrame
df_pos.to_parquet(f'{data_folder}ncr_training_redux_cleaned.parquet', index=False)
df_neg.to_parquet(f'{data_folder}pcg_training_redux_cleaned.parquet', index=False)

In [47]:
df.to_parquet(f'{data_folder}training_set_redux.parquet', index=False)

In [48]:
X = df.drop(columns=[label])
y = df[label]

In [49]:
# Train-test-validation split
# 70% train, 15% validation, 15% test

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, stratify=y, random_state=99)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=99)

logger.info(f"Train set shape: {X_train.shape}, {y_train.shape}")
logger.info(f"Validation set shape: {X_val.shape}, {y_val.shape}")
logger.info(f"Test set shape: {X_test.shape}, {y_test.shape}")
# Check if the target variable is balanced
logger.info(f"Train set target variable distribution: {y_train.value_counts(normalize=True)}")
logger.info(f"Validation set target variable distribution: {y_val.value_counts(normalize=True)}")
logger.info(f"Test set target variable distribution: {y_test.value_counts(normalize=True)}")

2025-07-29 00:30:25,744 [INFO] Train set shape: (23034, 36), (23034,)
2025-07-29 00:30:25,745 [INFO] Validation set shape: (4936, 36), (4936,)
2025-07-29 00:30:25,745 [INFO] Test set shape: (4936, 36), (4936,)
2025-07-29 00:30:25,747 [INFO] Train set target variable distribution: y
False   0.91
True    0.09
Name: proportion, dtype: float64
2025-07-29 00:30:25,748 [INFO] Validation set target variable distribution: y
False   0.91
True    0.09
Name: proportion, dtype: float64
2025-07-29 00:30:25,749 [INFO] Test set target variable distribution: y
False   0.91
True    0.09
Name: proportion, dtype: float64
2025-07-29 00:30:25,745 [INFO] Validation set shape: (4936, 36), (4936,)
2025-07-29 00:30:25,745 [INFO] Test set shape: (4936, 36), (4936,)
2025-07-29 00:30:25,747 [INFO] Train set target variable distribution: y
False   0.91
True    0.09
Name: proportion, dtype: float64
2025-07-29 00:30:25,748 [INFO] Validation set target variable distribution: y
False   0.91
True    0.09
Name: proporti

In [50]:
# Save the train, validation, and test sets
pd.concat([y_train, X_train], axis=1).to_parquet(f'{data_folder}X_train_redux.parquet', index=True)
pd.concat([y_val, X_val], axis=1).to_parquet(f'{data_folder}X_val_redux.parquet', index=True)
pd.concat([y_test, X_test], axis=1).to_parquet(f'{data_folder}X_test_redux.parquet', index=True)

# Evaluate which model algorithms should be considered for further testing

In [51]:
# from lazypredict.Supervised import LazyClassifier

# # Split the data into Train, Test and Validation sets
# # 70% for training, 15% for testing, 15% for validation
# X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
# X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)
# logger.info(f"Train set shape: {X_train.shape}, Test set shape: {X_test.shape}, Validation set shape: {X_val.shape}")

In [52]:
# df_lz = LazyClassifier(predictions=True, custom_metric=precision_score)
# # Fit the model
# models, predictions = df_lz.fit(X_train, X_test, y_train, y_test)

In [53]:
# models

In [54]:
# models.to_csv('lazypredict_redux.csv')

# Imbalanced training set
Since there are a much less positive instances (lncRNAs) than negative instances (protein coding) we can
1. Oversample the minority class -- lncRNAs
2. Undersample the majority class -- protein coding
3. Use imbalanced-resistant machine-learning algorithms (Random Forests) - tested during optimizations

## 1. Use SMOTE to oversample lncRNA instances

In [55]:
# Perform oversampling of the minority class
smote = SMOTE(random_state=99)
train_df = pd.read_parquet(f'{data_folder}X_train_redux.parquet')
label = 'y'
X_smote, y_smote = smote.fit_resample(train_df.drop(columns=[label]), train_df[label])
logger.info(f"Resampled dataset shape: {X_smote.shape}, {y_smote.shape}")

# Save the resampled data
pd.concat([y_smote, X_smote], axis=1).to_parquet(f'{data_folder}X_train_redux_smote.parquet', index=True)

2025-07-29 00:30:25,916 [INFO] Resampled dataset shape: (41894, 36), (41894,)


## 2. Oversampling and Undersample

In [56]:
# Use SMOTETomek to balance the dataset

smote_tomek = SMOTETomek(random_state=99)
train_df = pd.read_parquet(f'{data_folder}X_train_redux.parquet')
label = 'y'
X_smote_tomek, y_smote_tomek = smote_tomek.fit_resample(train_df.drop(columns=[label]), train_df[label])
logger.info(f"Resampled dataset shape: {X_smote_tomek.shape}, {y_smote_tomek.shape}")

# Save the resampled data
pd.concat([y_smote_tomek, X_smote_tomek], axis=1).to_parquet(f'{data_folder}X_train_redux_smote_tomek.parquet', index=True)


2025-07-29 00:30:26,664 [INFO] Resampled dataset shape: (41466, 36), (41466,)
